# Load modules

In [ ]:
#default_exp graphmodel

In [1]:
from fastai.tabular.all import *
from fastai.data.core import DataLoader
from fastcore.foundation import *
from fastcore.basics import *
from concurrent.futures import ProcessPoolExecutor
from torch import nn

### Pytorch layer

In [2]:
#export
class GCNLayer(Module):
    """
    Prototype Graph convolutional layer as described by Kipf & Welling(https://arxiv.org/pdf/1609.02907.pdf)
    
    """
    def __init__(self, ni=None, no=None):
        self.in_features = ni
        self.out_features = no
        self.lin = nn.Linear(ni,no)
        
    def forward(self, x):

        node_features, edge_features, adj_norm, degree_matrix = x # Get input features        
        updated_nodes = F.relu(self.lin(degree_matrix@adj_norm@degree_matrix@node_features))
        return updated_nodes, edge_features, adj_norm, degree_matrix

    def __repr__(self):
        return f'{self.__class__.__name__}\nInput features: {self.in_features} -> Output features: {self.out_features}' 

In [3]:
#export
class GCNModel(Module):
    def __init__(self, ni, nh, no, y_range):
        self.gcn = GCNLayer(ni, nh)
        self.graph_conv = nn.Sequential(GCNLayer(ni, nh), 
                                       GCNLayer(nh, nh),
                                       GCNLayer(nh, nh))

        self.mlp = nn.Sequential(
          nn.Linear(nh, nh//2),
            nn.Dropout(0.15),
          nn.ReLU(),
          nn.Linear(nh//2,nh//4),
            nn.Dropout(0.15),
          nn.ReLU())
        self.out = nn.Linear(nh//4, no)
        
        self.sigmoid = SigmoidRange(*y_range)
        
    def forward(self, x):
        x = self.graph_conv(x)
        x_lin = self.mlp(torch.mean(x[0],1))
        out = self.out(x_lin)
        return self.sigmoid(out)